In [1]:
pip install requests beautifulsoup4 faiss-cpu numpy sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import requests
from bs4 import BeautifulSoup
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
from transformers import pipeline
import re
import os

# Initialize models
model = SentenceTransformer("all-MiniLM-L6-v2")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

BASE_URL = "https://devgan.in"
LAW_TYPE = "bns"

# FAISS setup
d = 384  # Dimension of embeddings
index = faiss.IndexFlatL2(d)

# Ensure output directory exists
output_dir = "/content/Legal_AEye-Opener"
os.makedirs(output_dir, exist_ok=True)

def extract_sections(content):
    """
    Extracts individual sections while keeping IPC references inside the BNS section.
    Returns a list of dictionaries containing section_number, title, and content.
    """
    sections = []

    # DEBUG: Print raw content to analyze structure
    print("🔍 Checking raw chapter content:\n", content[:500])  # Print first 500 chars

    # Use regex to correctly identify BNS sections (Avoid treating IPC references as sections)
    section_pattern = re.split(r"(?=Section\s+\d+\s*–)", content)  # Match "Section X –" format

    for sec in section_pattern:
        match = re.match(r"Section\s+(\d+)\s*–\s*(.*)", sec.strip())
        if match:
            section_number = match.group(1)
            section_title = match.group(2)
            section_content = sec.strip()
        else:
            continue  # Skip if it doesn't match a section

        # Keep IPC references inside the same section
        sections.append({
            "section_number": section_number,
            "section_title": section_title,
            "section_content": section_content
        })

    return sections

def summarize_content(content):
    """Summarizes the given content."""
    return summarizer(content[:1024], max_length=150, min_length=50, do_sample=False)[0]['summary_text']

# Fetch the BNS chapters
MAIN_URL = f"{BASE_URL}/{LAW_TYPE}/"
response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, "html.parser")

chapters = []
for row in soup.select("table.menu tr"):
    columns = row.find_all("td")
    if len(columns) == 2:
        chapter_number = columns[0].text.strip()
        chapter_title = columns[1].text.strip()
        chapter_link = BASE_URL + columns[1].find("a")["href"]
        chapters.append((chapter_number, chapter_title, chapter_link))

# Scrape each chapter separately
for chapter_number, chapter_title, chapter_link in chapters:
    print(f"📥 Fetching Chapter {chapter_number}: {chapter_title}")

    chapter_response = requests.get(chapter_link)
    chapter_soup = BeautifulSoup(chapter_response.text, "html.parser")

    content_div = chapter_soup.find("div", id="content")
    if not content_div:
        print(f"⚠️ Warning: Could not find content div in Chapter {chapter_number}. Trying fallback method...")
        content_div = chapter_soup.find("div", class_="main-content")

    if content_div:
        chapter_content = content_div.get_text(separator="\n", strip=True)
    else:
        print(f"❌ Error: No content found for Chapter {chapter_number}")
        chapter_content = "Content not found."

    # Extract sections (IPC references remain inside BNS sections)
    sections = extract_sections(chapter_content)

    chapter_data = []
    for section in sections:
        section_summary = summarize_content(section["section_content"])
        formatted_summary = f"Section {section['section_number']}: {section_summary}" if section["section_number"] != "Unknown" else section_summary

        # Generate embedding and store in FAISS
        embedding = model.encode(section["section_content"]).astype(np.float32)
        index.add(np.array([embedding]))

        chapter_data.append({
            "law_type": LAW_TYPE.upper(),
            "chapter_number": chapter_number,
            "chapter_title": chapter_title,
            "section_number": section["section_number"],
            "section_title": section["section_title"],
            "section_content": section["section_content"],
            "section_summary": formatted_summary,
            "source_url": chapter_link
        })

        print(f"✅ Processed Chapter {chapter_number}, Section {section['section_number']}")

    # Save each chapter separately
    chapter_filename = f"{output_dir}/bns_chapter_{chapter_number}.json"
    with open(chapter_filename, "w", encoding="utf-8") as f:
        json.dump(chapter_data, f, ensure_ascii=False, indent=4)

    print(f"📂 Saved: {chapter_filename} ({len(chapter_data)} sections)")

# Save FAISS index
faiss.write_index(index, os.path.join(output_dir, "bns_faiss.index"))

print("🎉 BNS Laws Scraping Completed Successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ Processed Chapter 18, Section 347
✅ Processed Chapter 18, Section 348
✅ Processed Chapter 18, Section 349
✅ Processed Chapter 18, Section 350
📂 Saved: /content/Legal_AEye-Opener/bns_chapter_18.json (16 sections)
📥 Fetching Chapter 19: Criminal Intimidation, Insult, Annoyance, Defamation, Etc.
🔍 Checking raw chapter content:
 Chapter
XIX
– Of Criminal Intimidation, Insult, Annoyance, Defamation, Etc.
Section
351
– Criminal intimidation.
Whoever threatens by any means, another with any injury to his person, reputation or property, or to the person or reputation of any one in whom that person is interested, with intent to cause alarm to that person, or to cause that person to do any act which he is not legally bound to do, or to omit to do any act which that person is legally entitled to do, as the means of avoiding th


Your max_length is set to 150, but your input_length is only 124. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)


✅ Processed Chapter 19, Section 351
✅ Processed Chapter 19, Section 352
✅ Processed Chapter 19, Section 353


Your max_length is set to 150, but your input_length is only 149. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)


✅ Processed Chapter 19, Section 354
✅ Processed Chapter 19, Section 355
✅ Processed Chapter 19, Section 356
✅ Processed Chapter 19, Section 357
📂 Saved: /content/Legal_AEye-Opener/bns_chapter_19.json (7 sections)
📥 Fetching Chapter 20: Repeal And Savings
🔍 Checking raw chapter content:
 Chapter XX – Repeal And Savings
Section
358
– Repeal and savings.
The Indian Penal Code is hereby repealed.
Notwithstanding the repeal of the Code referred to in sub-section (1), it shall not affect,
the previous operation of the Code so repealed or anything duly done or suffered thereunder; or
any right, privilege, obligation or liability acquired, accrued or incurred under the Code so repealed; or
any penalty, or punishment incurred in respect of any offences committed against the Code so repe
✅ Processed Chapter 20, Section 358
📂 Saved: /content/Legal_AEye-Opener/bns_chapter_20.json (1 sections)
🎉 BNS Laws Scraping Completed Successfully!
